## Python 3 Notebook
### Libraries Used

In [1]:
import pandas as pd

### Daily Volatility Estimates

In [2]:
def get_daily_vol(close, span=100):
    """
    computes the daily volatility at intraday estimation points applying
    a 'span' of days to an exponentionally weighted moving standard deviation
    
    Args:
    
    Returns:
    
    """
    
    # daily vol, reindex to close
    df0 = close.index.searchsorted(close.index-pd.Timedelta(days=1))
    df0 = df0[df0>0]
    df0 = pd.Series(close.index[df0-1], index=close.index[close.shape[0]-df0.shape[0]:])
    df0 = close.loc[df0.index]/close.loc[df0.values].values-1 # daily returns
    df0 = df0.ewm(span=span).std()
    return df0

## Three Barrier Method
- Profit Taking, Stop Loss, and Time Limit

In [ ]:
def apply_pt_sl_on_tl(close, events, pt_sl, molecule):
    """
    apply profit taking / stop loss if it takes place before time limit
    
    Args:
        close (series): a pandas series of prices
        events (dataframe[['tl', 'trgt']]): a dataframe with horizontal and vertival barriers defined
        pt_sl (list): a list of two non-negative float values
        molecule (list): a list with the subset of event indices that will be processed by a single thread

    Returns:
        
    
    """
    
    events_ = events.loc[molecule]
    out = events_[['tl']].copy(deep=True)
    
    if pt_sl[0]>0:pt=pt_sl[0]*events_['trgt']
    else:pt=pd.Series(index=events.index) # NaNs

def apply_pt_sl_on_tl(close, events, pt_sl, molecule):
    """
    apply profit taking / stop loss if it takes place before time limit
    
    Args:
        close (series): a pandas series of prices
        events (dataframe[['tl', 'trgt']]): a dataframe with horizontal and vertival barriers defined
        pt_sl (list): a list of two non-negative float values
        molecule (list): a list with the subset of event indices that will be processed by a single thread

    Returns:
        
    
    """
    
    events_ = events.loc[molecule]
    out = events_[['tl']].copy(deep=True)
    
    if pt_sl[0]>0:pt=pt_sl[0]*events_['trgt']
    else:pt=pd.Series(index=events.index) # NaNs
        
    if pt_sl[1]>0:sl=-pt_sl[0]*events_['trgt']
    else:sl=pd.Series(index=events.index) # NaNs    
        
    for loc, tl in events_['tl'].fillna(close.index[-1]).iteritems():
        df0 = close[loc:tl] # path prices
        df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side'] # path returns
        out.loc[loc, 'sl'] = df0[df0<sl[loc]].index.min() # earliest stop lost
        out.loc[loc, 'pt'] = df0[df0>pt[loc]].index.min() # earliest profit taking
    
    return out
    

### First Barrier Touched